In [4]:
import numpy as np


def preprocessing(x, y):
    shf = list(range(len(x)))
    np.random.shuffle(shf)
    x_mix = []
    y_mix = []
    for i in range(len(x)):
        a = (x[i], y[i])
        b = (x[shf[i]], y[shf[i]])
        xc, yc = cutmix(a, b)
        x_mix.append(xc)
        y_mix.append(yc)
    return np.asarray(x_mix), np.asarray(y_mix)